<a href="https://colab.research.google.com/github/tractorjuice/wardley-maps-ai/blob/main/%5Cwardley_mapping_tools%5CConvert_WM_to_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ast import Index
import json
import re
import requests

def swap_xy(xy):
  new_xy = re.findall("\[(.*?)\]", xy)
  if new_xy:
    match = new_xy[0]
    match = match.split(sep = ",")
    match = match[::-1]
    new_xy = ('[' + match[0].strip() + ',' + match[1] + ']')
    return (new_xy)
  else:
    new_xy=""
    return (new_xy)

def parse_wardley_map(map_text):
    lines = map_text.strip().split("\n")
    title, evolution, anchors, components, nodes, links, evolve, pipelines, pioneers, market, blueline, notes, annotations, comments, style = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
    current_section = None

    for line in lines:
        if line.startswith("//"):
            comments.append(line)

        elif line.startswith("evolution"):
            evolution.append(line)

        elif "+<>" in line:
            blueline.append(line)

        elif line.startswith("title"):
            name = ' '.join(line.split()[1:])
            title.append(name)

        elif line.startswith("anchor"):
            name = line[line.find(' ') + 1:line.find('[')].strip()
            anchors.append(name)

        elif line.startswith("component"):
            stage = ""
            pos_index = line.find("[")
            if pos_index != -1:
                new_c_xy = swap_xy(line)
                number = json.loads(new_c_xy)
                if 0 <= number[0] <= 0.17:
                    stage = "genesis"
                elif 0.18 <= number[0] <= 0.39:
                    stage = "custom"
                elif 0.31 <= number[0] <= 0.69:
                    stage = "product"
                elif 0.70 <= number[0] <= 1.0:
                    stage = "commodity"
                else:
                    visibility = ""
                if 0 <= number[1] <= 0.20:
                    visibility = "low"
                elif 0.21 <= number[1] <= 0.70:
                    visibility = "medium"
                elif 0.71 <= number[1] <= 1.0:
                    visibility = "high"
                else:
                    visibility = ""               
            else:
                new_c_xy = ""

            name = line[line.find(' ') + 1:line.find('[')].strip()

            label_index = line.find("label")
            if label_index != -1:
                label = line[label_index+len("label")+1:]
                label = swap_xy(label)
            else:
                label = ""

            components.append({"name": name, "description": "", "evolution": stage, "visibility": visibility, "positionxy": new_c_xy, "labelxy": label})

        elif line.startswith("pipeline"):
            new_c_xy = swap_xy(line)
            name = line[line.find(' ') + 1:line.find('[')].strip()
            pipelines.append({"name": name, "description": "", "positionxy": new_c_xy, "labelxy": ""})

        elif line.startswith("links"):
            links.append(line)

        elif line.startswith("evolve"):
            new_c_xy = swap_xy(line)
            name = re.findall(r'\b\w+\b\s(.+?)\s\d', line)[0]
            label_index = line.find("label")
            if label_index != -1:
                label = line[label_index+len("label")+1:]
            else:
                label = ""
            label = swap_xy(label)
            evolve.append({"name": name, "description": "", "positionxy": new_c_xy, "labelxy": label})

        elif line.startswith("pioneer"):          
            pioneers.append(line)

        elif line.startswith("note"):
            name = line[line.find(' ') + 1:line.find('[')].strip()
            pos_index = line.find("[")
            if pos_index != -1:
                new_c_xy = swap_xy(line)
            else:
                new_c_xy = ""
            notes.append({"name": name, "description": "", "positionxy": new_c_xy, "labelxy": ""})   
                  
        elif line.startswith("annotations"):
            new_c_xy = swap_xy(line)
            annotations.append({"name": line, "description": "", "positionxy": new_c_xy})

        elif line.startswith("annotation"):
            new_c_xy = swap_xy(line)
            number = re.findall(r'\d+', line)
            name = line[line.index(']')+1:].lstrip()
            annotations.append({"number": number[0], "name": name, "description": "", "positionxy": new_c_xy})

        elif line.startswith("market"):
            name = line[line.find(' ') + 1:line.find('[')].strip()
            new_c_xy = swap_xy(line)
            label_index = line.find("label")
            if label_index != -1:
                label = line[label_index+len("label")+1:]
            else:
                label = ""
            label = swap_xy(label)
            market.append({"name": name, "description": "", "positionxy": new_c_xy, "labelxy": label})

        elif line.startswith("style"):
            style.append(line)

        elif "->" in line:
            source, target = line.strip().split("->")
            source = source.strip()
            target = target.strip()
            links.append({"source": source, "target": target})
        else:
            continue

    return {
        "title" : title,
        "anchors" : anchors,
        "evolution" : evolution,
        "components": components,
        "links": links,
        "evolve": evolve,
        "markets": market,
        "pipelines": pipelines,
        "pioneers": pioneers,
        "notes": notes,
        "blueline": blueline,
        "style": style,
        "annotations": annotations,
        "comments": comments,
    }

map_id='mUJtoSmOfqlfXhNMJP' 
url = f"https://api.onlinewardleymaps.com/v1/maps/fetch?id={map_id}"
response = requests.get(url)

# Check if the map was found
if response.status_code == 200:
    map_data = response.json()
    wardley_map_text = map_data['text']

# Parse the Wardley map text
    parsed_map = parse_wardley_map(wardley_map_text)

# Convert the parsed map to JSON
    wardley_map_json = json.dumps(parsed_map, indent=2)

# Print the JSON
    print(wardley_map_json)


{
  "title": [
    "Prompt Engineering"
  ],
  "anchors": [
    "Prompt Engineer"
  ],
  "evolution": [
    "evolution genesis / concept  -> custom / emerging  -> product / converging  -> commodity / accepted"
  ],
  "components": [
    {
      "name": "Techniques",
      "description": "",
      "evolution": "product",
      "visibility": "high",
      "positionxy": "[0.54,0.75]",
      "labelxy": "[-16,-38]"
    },
    {
      "name": "Chunks",
      "description": "",
      "evolution": "product",
      "visibility": "medium",
      "positionxy": "[0.62,0.39]",
      "labelxy": "[23,-17]"
    },
    {
      "name": "Text Splitter",
      "description": "",
      "evolution": "product",
      "visibility": "medium",
      "positionxy": "[0.67,0.34]",
      "labelxy": ""
    },
    {
      "name": "Embedding",
      "description": "",
      "evolution": "product",
      "visibility": "medium",
      "positionxy": "[0.59,0.45]",
      "labelxy": ""
    },
    {
      "name": "Tools",
 